In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from PIL import Image
import numpy as np

# Ana veri seti klasörünüz
dataset_dir = "dataset"
haze_dir = os.path.join(dataset_dir, "haze")
clear_dir = os.path.join(dataset_dir, "clear")


In [ ]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
from PIL import Image

# dataset klasörünü bağlama
dataset_path = '/content/drive/MyDrive/dataset'
#os.symlink(dataset_path, '/content/dataset')  # Sembolik bağlantı
working_dir = '/content/dataset'

# Alt klasörler
haze_dir = os.path.join(dataset_path, 'haze')
clear_dir = os.path.join(dataset_path, 'clear')

In [ ]:
import os
print("Çalışma dizini:", os.getcwd())

Çalışma dizini: /content


In [ ]:
# Haze ve clear klasörlerindeki tüm görüntüleri listeleme
haze_images = [os.path.join(haze_dir, fname) for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]
clear_images = [os.path.join(clear_dir, fname) for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]

# İlk birkaç görüntüyü kontrol etmek için yükleyip gösterelim
for img_path in haze_images[:5]:
    img = Image.open(img_path)
    img.show()


In [ ]:
import numpy as np
from PIL import Image
from multiprocessing import Pool

# Görüntüleri yükleme ve ön işleme fonksiyonu
def load_and_preprocess_image(img_path, target_size=(256, 256)):  # Boyutlandırmayı 256'e düşürdüm
    try:
        print(f"İşleniyor: {img_path}")  # Loglama eklendi
        img = Image.open(img_path).resize(target_size)  # Yeniden boyutlandırma
        img = np.array(img) / 255.0  # Normalizasyon
        return img
    except Exception as e:
        print(f"Görüntü yüklenemedi: {img_path}, Hata: {e}")
        return None

# Paralel işleme ile görüntüleri yükleme fonksiyonu
def load_images_parallel(image_paths, num_workers=4):
    with Pool(processes=num_workers) as pool:
        images = pool.map(load_and_preprocess_image, image_paths)

    # None dönen hatalı görüntüleri filtreleme
    images = [img for img in images if img is not None]
    return np.array(images)

# Alt küme ile test etmek isterseniz:
haze_images_subset = haze_images[:50]  # İlk 50 sisli görüntüyü al
clear_images_subset = clear_images[:50]  # İlk 50 temiz görüntüyü al

# Haze görüntüleri yükleme
print("Haze görüntüleri yükleniyor...")
haze_data = load_images_parallel(haze_images_subset, num_workers=4)  # 4 çekirdek kullanılıyor
print(f"Haze görüntüleri yüklendi, toplam {len(haze_data)} adet görüntü işleme alındı.")

# Clear görüntüleri yükleme
print("Clear görüntüleri yükleniyor...")
clear_data = load_images_parallel(clear_images_subset, num_workers=4)
print(f"Clear görüntüleri yüklendi, toplam {len(clear_data)} adet görüntü işleme alındı.")

# Yüklenen veri kümelerinin boyutlarını kontrol et
print(f"Haze veri şekli: {haze_data.shape}")
print(f"Clear veri şekli: {clear_data.shape}")



Haze görüntüleri yükleniyor...
İşleniyor: 8829_1_0.04.jpgİşleniyor: 8824_0.9_0.06.jpgİşleniyor: 8825_0.8_0.1.jpg

İşleniyor: 8818_0.95_0.12.jpgGörüntü yüklenemedi: 8824_0.9_0.06.jpg, Hata: [Errno 2] No such file or directory: '/content/8824_0.9_0.06.jpg'Görüntü yüklenemedi: 8829_1_0.04.jpg, Hata: [Errno 2] No such file or directory: '/content/8829_1_0.04.jpg'

Görüntü yüklenemedi: 8825_0.8_0.1.jpg, Hata: [Errno 2] No such file or directory: '/content/8825_0.8_0.1.jpg'

İşleniyor: 8825_0.9_0.2.jpg
Görüntü yüklenemedi: 8818_0.95_0.12.jpg, Hata: [Errno 2] No such file or directory: '/content/8818_0.95_0.12.jpg'
İşleniyor: 8829_0.8_0.06.jpgİşleniyor: 8823_0.95_0.06.jpg
Görüntü yüklenemedi: 8825_0.9_0.2.jpg, Hata: [Errno 2] No such file or directory: '/content/8825_0.9_0.2.jpg'
İşleniyor: 8823_0.9_0.08.jpg
Görüntü yüklenemedi: 8829_0.8_0.06.jpg, Hata: [Errno 2] No such file or directory: '/content/8829_0.8_0.06.jpg'
İşleniyor: 8827_0.8_0.04.jpg

Görüntü yüklenemedi: 8823_0.9_0.08.jpg, Hata:

In [ ]:
import os

# Dataset yolunu kontrol et
dataset_path = '/content/drive/MyDrive/dataset'
clear_dir = os.path.join(dataset_path, "clear")
haze_dir = os.path.join(dataset_path, "haze")

# Klasörlerin varlığını doğrula
if not os.path.exists(clear_dir):
    raise FileNotFoundError(f"'clear' klasörü bulunamadı: {clear_dir}")
if not os.path.exists(haze_dir):
    raise FileNotFoundError(f"'haze' klasörü bulunamadı: {haze_dir}")

# Clear klasöründeki tüm dosyaların isimlerini alalım
clear_images = [fname for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]

# Haze klasöründeki tüm dosyaların isimlerini alalım
haze_images = [fname for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]

# Clear görüntüsüne karşılık gelen haze görüntülerini eşleştireceğimiz bir sözlük oluşturuyoruz
haze_images_dict = {clear_img: [] for clear_img in clear_images}

# Clear görüntülerine karşılık gelen haze görüntülerini bulalım
for clear_img in clear_images:
    # Clear görüntüsünün dosya adı (uzantısız)
    base_name = os.path.splitext(clear_img)[0]

    # Bu base_name'e sahip tüm haze dosyalarını buluyoruz
    corresponding_haze = [haze_img for haze_img in haze_images if haze_img.startswith(base_name)]

    # Eğer haze görüntüleri varsa, clear görüntüsüne karşılık gelen listeye ekliyoruz
    if corresponding_haze:
        for haze_img in corresponding_haze:
            haze_images_dict[clear_img].append(os.path.join(haze_dir, haze_img))

# Sonuçları kontrol edelim
for clear_img, haze_list in haze_images_dict.items():
    print(f"{clear_img} için {len(haze_list)} haze görüntüsü bulundu:")
    for haze_img in haze_list:
        print(f"    - {haze_img}")


Görüntülenen çıkış son 5000 satıra kısaltıldı.
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.06.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.95_0.08.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.04.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.16.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.2.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.95_0.04.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.2.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.06.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.85_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.8_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.9_0.04.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.9_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.9_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/3168_0.95_

In [ ]:
# 0 haze resmi olan clear görüntüleri bulalım
zero_haze_clear_images = [clear_img for clear_img, haze_list in haze_images_dict.items() if len(haze_list) == 0]

# Sonuçları yazdıralım
if zero_haze_clear_images:
    print("0 haze resmi olan clear görüntüler:")
    for clear_img in zero_haze_clear_images:
        print(f"    - {clear_img}")
else:
    print("Tüm clear görüntüler için en az bir haze görüntüsü var.")

print(len(zero_haze_clear_images))

Haze'i bulunmayan clear data dosya adları: -2949.jpg - 2946.jpg - 2989.jpg - 2831.jpg - 2856.jpg - 2935.jpg - 2675.jpg - 2979.jpg - 2881.jpg - 2980.jpg - 2676.jpg - 2954.jpg - 2997.jpg - 2994.jpg - 2864.jpg - 2868.jpg - 2981.jpg - 2650.jpg - 2834.jpg - 2647.jpg - 2993.jpg - 2990.jpg - 2880.jpg - 2869.jpg - 2867.jpg - 2944.jpg - 2971.jpg - 2938.jpg - 2945.jpg - 2988.jpg - 2863.jpg - 2848.jpg - 2943.jpg - 2836.jpg - 2973.jpg - 2649.jpg - 2941.jpg - 2992.jpg - 2982.jpg - 2947.jpg - 2854.jpg - 2679.jpg - 2986.jpg - 2995.jpg - 2860.jpg - 2975.jpg - 2855.jpg - 2948.jpg - 2853.jpg - 2756.jpg - 2865.jpg - 2978.jpg - 2977.jpg - 2866.jpg - 2976.jpg - 2987.jpg - 2646.jpg - 2665.jpg - 2985.jpg - 2974.jpg - 2984.jpg - 2999.jpg - 2754.jpg - 2858.jpg - 2837.jpg - 2970.jpg - 2861.jpg - 2972.jpg - 2998.jpg - 3062.jpg - 3065.jpg - 3001.jpg - 3059.jpg - 3014.jpg - 3004.jpg - 3064.jpg - 3066.jpg - 3002.jpg - 3040.jpg - 3058.jpg - 3007.jpg - 3028.jpg - 3009.jpg - 3054.jpg - 3008.jpg